In [7]:
#Data access, data processing
import json
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
#Choropleth
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap
from folium import plugins
with open('region.json') as f:
     regions = json.load(f)

In [8]:
ll = pd.read_csv('tc.csv')
ll.sort_values(by=['Region'],inplace=True)
dn = pd.read_csv('../datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
start_date,end_date='2017-01-01','2018-01-01'
years = dn[(dn.index>=start_date)&(dn.index<end_date)]
years = years.T
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
years = pd.DataFrame(min_max_scaler.fit_transform(years),columns=years.columns, index=years.index) 
years.sort_index(inplace=True)
years.index = ll.Region
years.reset_index()
years = years.T
years.columns = [str(col) + '_M' for col in years.columns]
for cols in years.columns:
    region = cols.replace('_M', '')
    years[region+'_La'] = ll.loc[ll['Region'] == region, 'Latitude'].iloc[0]
    years[region+'_Lo'] = ll.loc[ll['Region'] == region, 'Longitude'].iloc[0]
years = years.reindex(sorted(years.columns), axis=1)
time_index = years.index.to_series().apply(lambda x: datetime.strftime(x, '%Y-%m-%d')).tolist()
data = (np.array(years).reshape(-1,3))

In [9]:
region_geo = os.path.join('.', 'region.json')
for idx,month  in  enumerate(time_index):
    df = pd.DataFrame(data[13*idx:(13*idx)+13],index=ll.Region, columns=['Latitude','Longitude','Magnitude'])
    df['Code'] = ll['Code'].values
    df['Code'] = df['Code'].astype(str)
    m = Map(location=[46.9, 2.], zoom_start=6)
    m.choropleth(
     name='choropleth',
     geo_data=region_geo,
     data=df,
     columns=['Code', 'Magnitude'],
     key_on='feature.properties.code',
     fill_color='YlOrRd',
     fill_opacity=0.7,
     line_opacity=0.2,
     reset = True,
     legend_name='Demande Touristique'
    ) 
    LayerControl().add_to(m)
    m.save('choropleth-'+str(month)+'.html')


In [ ]:
import os
import time
from selenium import webdriver

delay=5
for idx, month  in  enumerate(time_index):
    fn = 'choropleth-'+month+'.html'
    tmpurl ='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    browser = webdriver.Firefox()
    browser.get(tmpurl)
    time.sleep(delay)
    browser.save_screenshot(month+'.png')
    browser.quit()